In [1]:
!pwd

/home/techie/Desktop/general/End-to-End-DL/notebooks


In [2]:
import os

os.chdir("../")
!pwd

/home/techie/Desktop/general/End-to-End-DL


In [ ]:
from PnemoniaDisease.utils.utils import read_yaml, create_directories
from PnemoniaDisease import logger
from PnemoniaDisease.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from pathlib import Path
import tensorflow as tf

import os
import time

In [62]:
class PrepareCallback:
    def __init__(self,config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)      
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.prepare_callbacks.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.prepare_callbacks.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [63]:
class Training(PrepareCallback):
    def __init__(self,
                 params_filepath=PARAMS_FILE_PATH,
                 config_filepath=CONFIG_FILE_PATH):
        super().__init__(params_filepath)
        self.params = read_yaml(params_filepath)
        self.config = read_yaml(config_filepath)

        # create_directories([self.config.artifacts_root])

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.prepare_base_model.base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.params.IMAGE_SIZE[:-1],
            batch_size=self.params.BATCH_SIZE,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        training = self.config.training
        create_directories([
            Path(training.root_dir)
        ])
        training_data = os.path.join("data/processed", "train")
        

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path(training_data),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.params.AUGMENTATION:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=Path(training_data),
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list): 
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.params.EPOCHS,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.training.trained_model_path,
            model=self.model
        )

In [64]:
try:
    prepare_callbacks = PrepareCallback(config_filepath=CONFIG_FILE_PATH)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training = Training(config_filepath=CONFIG_FILE_PATH, 
                        params_filepath=PARAMS_FILE_PATH)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-07-30 13:03:07,164: INFO: utils: yaml file: config/config.yaml loaded successfully]
[2023-07-30 13:03:07,218: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-30 13:03:07,230: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-30 13:03:07,238: INFO: utils: yaml file: config/config.yaml loaded successfully]
[2023-07-30 13:03:07,241: INFO: utils: created directory at: artifacts]
[2023-07-30 13:03:23,466: INFO: utils: created directory at: artifacts/training]
Found 79 images belonging to 2 classes.
Found 322 images belonging to 2 classes.
20/20 [==============================] - ETA: 0s - loss: 0.7248 - accuracy: 0.6797

/home/techie/anaconda3/envs/dl-env/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 96s 4s/step - loss: 0.7248 - accuracy: 0.6797 - val_loss: 0.3897 - val_accuracy: 0.9219
